**Disclaimer:** some of the material is borrowed with thanks from http://www.cis.lmu.de/~hs/teach/14s/ir/

# Boolean model

* The Boolean model is arguably the simplest model to
  base an information retrieval system on.
* Queries are Boolean expressions, e.g., *Caesar* and *Brutus*
* The seach engine returns all documents that satisfy the Boolean expression, but there are exceptions:
  * anchor text
  * page contains variant of query words: morphology, spelling correction, synonyms
* No notion of ranking

## Term-document matrix

* Rows: terms
* Columns: document
* $M_{ij}=1$ if term *i* present in document *j*


In [1]:
from sklearn.feature_extraction.text import CountVectorizer
documents=["This is a silly example","A better example","Nothing to see here","This is a great and long example"]
cv=CountVectorizer(lowercase=False,binary=True)
print("Term-document matrix:\n")
td_matrix=cv.fit_transform(documents).todense().T   #.T transposes the matrix, sklearn maintains document-term
print(td_matrix)
print("\nIDX -> terms mapping:\n")
print(cv.get_feature_names())
print("\nterm -> IDX mapping:\n")
print(cv.vocabulary_) # note the _ at the end


Term-document matrix:

[[0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [1 1 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]]

IDX -> terms mapping:

['Nothing', 'This', 'and', 'better', 'example', 'great', 'here', 'is', 'long', 'see', 'silly', 'to']

term -> IDX mapping:

{'silly': 10, 'This': 1, 'and': 2, 'better': 3, 'is': 7, 'example': 4, 'here': 6, 'great': 5, 'see': 9, 'Nothing': 0, 'to': 11, 'long': 8}


* Every row is an *incidence vector* of a term - which documents the term appears in
* Boolean retrieval in its simplest form:
  * Boolean operations on incidence vectors

In [2]:
t2i=cv.vocabulary_
print("example")
print(td_matrix[t2i["example"]])
print()
print("example and great")
print(td_matrix[t2i["example"]] & td_matrix[t2i["great"]])
print()
print("not example")
print(1-td_matrix[t2i["example"]]) #1-x does the negation in our case
print()

example
[[1 1 0 1]]

example and great
[[0 0 0 1]]

not example
[[0 0 1 0]]



Now we can piece it all together:

* Accept queries like "( not example or great ) and Nothing"
* Rewrite them into a Python expression
* eval() that expression

In [3]:
# Operators and, or, not become &, |, 1 -
# Parentheses are left untouched
# Everything else interpreted as a term and fed through td_matrix[t2i["..."]]

d={"and":"&","or":"|","not":"1 -","(":"(",")":")"} # operator replacements
def rew_token(t):
    return d.get(t,'td_matrix[t2i["{}"]]'.format(t))

def rew_query(query): #rewrite every token in the query
    return " ".join(rew_token(t) for t in query.split())

def test_query(query):
    print("Query:\""+query+"\"")
    print("Rewritten:",rew_query(query))
    print("Matching:",eval(rew_query(query)))
    print()

test_query("example and not Nothing")
test_query("not example or great")
test_query("( not example or great ) and Nothing")


Query:"example and not Nothing"
Rewritten: td_matrix[t2i["example"]] & 1 - td_matrix[t2i["Nothing"]]
Matching: [[1 1 0 1]]

Query:"not example or great"
Rewritten: 1 - td_matrix[t2i["example"]] | td_matrix[t2i["great"]]
Matching: [[0 0 1 1]]

Query:"( not example or great ) and Nothing"
Rewritten: ( 1 - td_matrix[t2i["example"]] | td_matrix[t2i["great"]] ) & td_matrix[t2i["Nothing"]]
Matching: [[0 0 1 0]]



In [4]:
# Should match all documents, but crashes instead
# This one you will fix as an exercise
import traceback
try:
    test_query("not awordwhichdoesnotexist") #should match all documents
except:
    traceback.print_exc() #I only need to do this because of IPython

Query:"not awordwhichdoesnotexist"
Rewritten: 1 - td_matrix[t2i["awordwhichdoesnotexist"]]


Traceback (most recent call last):
  File "<ipython-input-4-69ec74878d51>", line 5, in <module>
    test_query("not awordwhichdoesnotexist") #should match all documents
  File "<ipython-input-3-41857662fdc6>", line 15, in test_query
    print("Matching:",eval(rew_query(query)))
  File "<string>", line 1, in <module>
KeyError: 'awordwhichdoesnotexist'


* We now have a rudimentary boolean IR system
* It is not that great but ready in < 10 lines of code in Python

# Size constraints

* Term-document matrix is vocabulary size times document set size
* 1M documents, each 1000 words
* 1,000,000 x 1,000 = 1,000,000,000 words of running text
* 6 bytes per word -> ~6GB in size
* Assume 500,000 unique terms
* Term-document matrix has 1,000,000 x 500,000 / 8 / 1024 / 1024 / 1024 -> ~60GB
* 60GB of space to index a collection of 6GB of text!
* 480GB if we were to use 1B integers to remember the 0/1 values
* ...but most of this are zeros...

## Sparse representation

* Only remember the non-zero entries
* For every term, remember a (usually sorted) list of documents in which it appears
* This is the famous **inverted index**
* Scipy sparse formats: https://docs.scipy.org/doc/scipy-0.18.1/reference/sparse.html
  * *CSC* for every column remember the list of rows
  * *CSR* for every row remember the list of columns
  


In [5]:
documents=["This is a silly example","A better example","Nothing to see here","This is a great and long example"]
cv=CountVectorizer(lowercase=False,binary=True)
# Exact same code as above, but I removed the .todense()
td_matrix=cv.fit_transform(documents)
print("scikit's document-term matrix:")
print(td_matrix)
print()
print("Transposed: (note incorrect sort)")
print(td_matrix.T)
print()
print("Transposed, and in the correct sparse format:")
print(td_matrix.T.tocsr())
td_matrix=td_matrix.T.tocsr()

scikit's document-term matrix:
  (0, 1)	1
  (0, 7)	1
  (0, 10)	1
  (0, 4)	1
  (1, 4)	1
  (1, 3)	1
  (2, 0)	1
  (2, 11)	1
  (2, 9)	1
  (2, 6)	1
  (3, 1)	1
  (3, 7)	1
  (3, 4)	1
  (3, 5)	1
  (3, 2)	1
  (3, 8)	1

Transposed: (note incorrect sort)
  (1, 0)	1
  (7, 0)	1
  (10, 0)	1
  (4, 0)	1
  (4, 1)	1
  (3, 1)	1
  (0, 2)	1
  (11, 2)	1
  (9, 2)	1
  (6, 2)	1
  (1, 3)	1
  (7, 3)	1
  (4, 3)	1
  (5, 3)	1
  (2, 3)	1
  (8, 3)	1

Transposed, and in the correct sparse format:
  (0, 2)	1
  (1, 0)	1
  (1, 3)	1
  (2, 3)	1
  (3, 1)	1
  (4, 0)	1
  (4, 1)	1
  (4, 3)	1
  (5, 3)	1
  (6, 2)	1
  (7, 0)	1
  (7, 3)	1
  (8, 3)	1
  (9, 2)	1
  (10, 0)	1
  (11, 2)	1


In [6]:
# The sparse representations do not allow many of the necessary operations, so we need
# to make the rows dense, once we retrieve them, not a huge deal for our toy examples
def rew_token(t):
    return d.get(t,'td_matrix[t2i["{}"]].todense()'.format(t))

test_query("example and not Nothing")
test_query("( not example or great ) and Nothing")

Query:"example and not Nothing"
Rewritten: td_matrix[t2i["example"]].todense() & 1 - td_matrix[t2i["Nothing"]].todense()
Matching: [[1 1 0 1]]

Query:"( not example or great ) and Nothing"
Rewritten: ( 1 - td_matrix[t2i["example"]].todense() | td_matrix[t2i["great"]].todense() ) & td_matrix[t2i["Nothing"]].todense()
Matching: [[0 0 1 0]]



## Boolean retrieval (cont.)

* The code above needs `.todense()` to perform the *and / or / not* arithmetics -> inefficient, but simple
* Other option - make sure the lists of documents are sorted
* AND - intersection of two sorted lists
  * Walk the lists (I'll show on the lecture how - it's rather obvious anyway)
  * Linear complexity in terms of number of documents
  * A good exercise - write a function which takes two lists like those above, and computes their intersection as a new list

In [7]:
print("Documents for 'example'")
print(td_matrix[t2i["example"]].nonzero()[1])
print()
print("Documents for 'great'")
print(td_matrix[t2i["great"]].nonzero()[1])

Documents for 'example'
[0 1 3]

Documents for 'great'
[3]


# Phrase and proximity queries

* *"Stanford university"* - not the same thing as *Stanford AND university*
* The basic inverted index of no help

## biword index

* Index all word bigrams
* "Stanford university" becomes a term
* "to be or not to be" -> "to be" and "be or" and "or not" and "not to" + postprocessing
* Not a great solution:
  * Massive waste of space: blows up the index size quadratically
  * Needs postprocessing to weed out the false hits

## positional index

* For every term, and every document, remember a list of the positions, not just "1"
* A modification of the sorted list intersection algorithm to answer the query
* Also allows proximity queries "X within N words from Y"
* Quite heavy computationally

## combined index

* Most common biwords indexed directly
* Rest solved with positional indexing
* Compromise between the two

# Result ranking

* Obviously useful for large document collections
* Come up with a number describing the fit of a document to a query
* Return top-N documents
* Basic observations:
  * The more query terms hit, the more relevant the document is
  * The more times the query terms hit in the document, the more relevant the document is
  * Rare terms are more informative than common terms
* Firstly, we don't want to store 0/1, we want to store the count

In [8]:
documents=["This is a silly silly silly example","A better example","Nothing to see here nor here nor here","This is a great example and a long example too"]
cv=CountVectorizer(lowercase=False)
# Exact same code as above, but now I also removed binary=True
td_matrix=cv.fit_transform(documents)
t2i=cv.vocabulary_
td_matrix=td_matrix.T.tocsr()
print(td_matrix.todense())

[[0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [1 1 0 2]
 [0 0 0 1]
 [0 0 3 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 2 0]
 [0 0 1 0]
 [3 0 0 0]
 [0 0 1 0]
 [0 0 0 1]]


* Sum up the counts across query hits

In [9]:
print("Documents for 'example' and 'better'")
hits=td_matrix[t2i["example"]]+td_matrix[t2i["better"]]
print("Hits:",hits,sep="\n")
print("Documents:", hits.nonzero()[1])
print("Scores:", hits[hits.nonzero()].A1) #A1 returns itself as flat array


Documents for 'example' and 'better'
Hits:
  (0, 0)	1
  (0, 1)	2
  (0, 3)	2
Documents: [0 1 3]
Scores: [1 2 2]


* Now we can sort on scores and get an order on documents
* Now a document with 2x example hits equally well as document with 1x example and 1x better
* Maybe not optimal?
* A document with a term occurring 10x is more important, but not ten times so
* Usual solution: squeeze the counts through log or similar function (done already at index time)
  * $1+log_{10}(tf)$ is a typical formula used

In [10]:
# Switch from CountVectorizer to TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Parameters with which TfidfVectorizer does same thing as CountVectorizer
tfv=TfidfVectorizer(lowercase=False,sublinear_tf=False,use_idf=False,norm=None)
td_matrix=tfv.fit_transform(documents)
t2i=cv.vocabulary_
td_matrix=td_matrix.T.tocsr()
print("Just to check we get same numbers as with CountVectorizer:")
print(td_matrix.todense())

# Turn log-squeeze on
tfv=TfidfVectorizer(lowercase=False,sublinear_tf=True,use_idf=False,norm=None)
td_matrix=tfv.fit_transform(documents)
t2i=cv.vocabulary_
td_matrix=td_matrix.T.tocsr()
print("Just to check we get same numbers as with CountVectorizer:")
print(td_matrix.todense())


Just to check we get same numbers as with CountVectorizer:
[[ 0.  0.  1.  0.]
 [ 1.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 1.  1.  0.  2.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  3.  0.]
 [ 1.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  2.  0.]
 [ 0.  0.  1.  0.]
 [ 3.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
Just to check we get same numbers as with CountVectorizer:
[[ 0.          0.          1.          0.        ]
 [ 1.          0.          0.          1.        ]
 [ 0.          0.          0.          1.        ]
 [ 0.          1.          0.          0.        ]
 [ 1.          1.          0.          1.69314718]
 [ 0.          0.          0.          1.        ]
 [ 0.          0.          2.09861229  0.        ]
 [ 1.          0.          0.          1.        ]
 [ 0.          0.          0.          1.        ]
 [ 0.          0.          1.69314718  0.        ]
 [ 0.          0.          1.          0.        ]
 [ 2.09861229  0.          0.          0.        ]
 [ 

* and now the *example* and *better* document should rank above the *2x example* document

In [11]:
print("Documents for 'example' and 'better'")
hits=td_matrix[t2i["example"]]+td_matrix[t2i["better"]]
print("Hits:",hits,sep="\n")
print("Documents:", hits.nonzero()[1])
print("Scores:", hits[hits.nonzero()].A1) #A1 returns itself as flat array

Documents for 'example' and 'better'
Hits:
  (0, 0)	1.0
  (0, 1)	2.0
  (0, 3)	1.69314718056
Documents: [0 1 3]
Scores: [ 1.          2.          1.69314718]


## Informativeness of terms

* Not all words are equally informative
* Rare words are clearly much more informative than common ones
* The query *"jabberwocky movie cast"* should put more weight on *jabberwocky*
* What we want:
  * High positive weight for rare terms
  * Low positive weight for common terms
* Typical way: IDF *inverse document frequency*
  * df_t is the *document frequency* of *t* - number of documents *t* appear in
  * $IDF_t=\frac{N}{df_t}$ inverse document frequency of *t*
  * Usually one would squeeze this through log so
  * $IDF_t=log_{10}(\frac{N}{df_t})$

## Tf.Idf

* An extremely common weighting scheme in IR
* Product of term's tf (log-squeezed) with the term's idf (log-squeezed)
* Gives a score of that term's hit in a given document
  * $(1+log_{10}tf_t)\cdot log_{10}\frac{N}{df_t}$
  

In [12]:
# same as above, but use_idf=True
tfv=TfidfVectorizer(lowercase=False,sublinear_tf=True,use_idf=True,norm=None)
td_matrix=tfv.fit_transform(documents)
t2i=cv.vocabulary_
td_matrix=td_matrix.T.tocsr()
print("Just to check we get same numbers as with CountVectorizer:")
print(td_matrix.todense())

Just to check we get same numbers as with CountVectorizer:
[[ 0.          0.          1.91629073  0.        ]
 [ 1.51082562  0.          0.          1.51082562]
 [ 0.          0.          0.          1.91629073]
 [ 0.          1.91629073  0.          0.        ]
 [ 1.22314355  1.22314355  0.          2.07096206]
 [ 0.          0.          0.          1.91629073]
 [ 0.          0.          4.02155128  0.        ]
 [ 1.51082562  0.          0.          1.51082562]
 [ 0.          0.          0.          1.91629073]
 [ 0.          0.          3.24456225  0.        ]
 [ 0.          0.          1.91629073  0.        ]
 [ 4.02155128  0.          0.          0.        ]
 [ 0.          0.          1.91629073  0.        ]
 [ 0.          0.          0.          1.91629073]]


In [13]:
print("Documents for 'example' and 'better' with full tf.idf weighting")
hits=td_matrix[t2i["example"]]+td_matrix[t2i["better"]]
print("Hits:",hits,sep="\n")
print("Documents:", hits.nonzero()[1])
print("Scores:", hits[hits.nonzero()].A1) #A1 returns itself as flat array

Documents for 'example' and 'better' with full tf.idf weighting
Hits:
  (0, 0)	1.22314355131
  (0, 1)	3.13943428319
  (0, 3)	2.07096205533
Documents: [0 1 3]
Scores: [ 1.22314355  3.13943428  2.07096206]


* Doesn't work on toy examples, really, let's index a bit more
* http://linguatools.org/tools/corpora/wikipedia-monolingual-corpora/

In [14]:
%%bash
zcat fiwiki-20140809-corpus.txt.gz | head -n 40

<article name="Amsterdam">
Amsterdam on Alankomaiden pääkaupunki. Amsterdam on väkiluvultaan Alankomaiden suurin kaupunki, huhtikuun alussa 2006 siellä asui 743 905 asukasta eli noin joka 20. hollantilainen asuu varsinaisessa Amsterdamissa. Yhteensä Amsterdamissa ja sitä ympäröivällä kaupunkialueella asuu noin 1 450 000 ihmistä eli vajaa kymmenesosa Alankomaiden asukkaista. Amsterdam sijaitsee Amstelin suistossa IJsselmeerin rannalla Alankomaiden Pohjois-Hollannin provinssissa. Vaikka Amsterdam on Alankomaiden perustuslain mukaan maan pääkaupunki, sijaitsee niin kuningashuone, hallitus, parlamentti kuin korkein oikeuskin Haagissa.
Historia
Amsterdamin paikalle rakennettiin ensimmäiset puutalot 1200-luvun alkupuolella. Kaupunki linnoitettiin suojaksi vihollisilta; Amstel-joki padottiin nykyisen Damin aukion kohdalta. Padon avulla pystyttiin säätelemään veden korkeuden lisäksi myös kauppaa, sillä se esti merikelpoisia aluksia purjehtimasta yläjuoksulle. Siksi rahti jouduttiinkin siirtämä

In [15]:
import gzip
def articles(gzipfile,max_articles=1000):
    """A function to yield documents, one at a time"""
    with gzip.open(gzipfile,"rt") as f:
        article=[]
        for line in f:
            line=line.strip()
            article.append(line)
            if line=="</article>":
                yield " ".join(article)
                max_articles-=1
                if max_articles==0:
                    break
                article=[]
# Index 50K articles from Wikipedia, turn on lowercasing too
tfv_wiki=TfidfVectorizer(input="content",lowercase=True,sublinear_tf=True,use_idf=True,norm=None)
td_matrix_wiki=tfv_wiki.fit_transform(articles("fiwiki-20140809-corpus.txt.gz",50000))

In [17]:
import numpy as np
# I will need access to which term is at which position
vocab_list=[None]*len(tfv_wiki.vocabulary_)
for term,idx in tfv_wiki.vocabulary_.items():
    vocab_list[idx]=term

# Indirect sort, returns array of indices
idf_sorted=np.argsort(tfv_wiki.idf_)
# top-30
for t_idx in idf_sorted[:30]:
    print(tfv_wiki.idf_[t_idx],"    ",vocab_list[t_idx])

1.0      name
1.0      article
1.11305432843      ja
1.19011569707      on
1.50000460039      myös
1.62696753965      oli
1.69136879842      joka
1.73190800868      vuonna
1.87296392717      se
1.90186556724      ovat
1.91177105163      ei
1.91546109272      sen
1.936105359      sekä
1.93799412651      mutta
2.06521685493      tai
2.11805978613      että
2.16342684238      hän
2.21951261578      jälkeen
2.23720493833      jonka
2.28766431015      mukaan
2.29932965095      kanssa
2.31403090933      kun
2.3273664381      jossa
2.33486107933      vuoden
2.34994385961      eli
2.35396334404      kuin
2.41561959531      kuitenkin
2.42190008864      muun
2.45879903595      muassa
2.47065294936      noin


# Vector-space model

* So far we looked at term vectors (rows of the term-document matrix)
* We could also think about document vectors (columns of the term-document matrix)
* Documents are vectors in a high-dimensional space, terms are the dimensions
* Document vectors very high-dimensional but very very sparse
* Same for queries - queries can also be seen as vectors in a high-dimensional space
* Search:
  * similarity between query vector and document vector
  * higher similarity means better hit (rank higher)
  
## Similarity measures

* Similarity -> negative distance
* Eucledian distance -> affected by vector length
* Cosine similarity -> cosine of the angle between query and document vectors
  * 1 for total similarity (angle 0)
  * -1 for complete opposite
  * we only have positive numbers in our vectors, so we are on the [0,1] scale not [-1,1]
  * not sensitive to length
  * incredibly easy to compute!
  * $cos(u,v)=\frac{u\cdot v}{||u||\cdot ||v||}=\frac{\sum_i v_i\cdot u_i}{\sqrt{\sum_i u_i^2}\sqrt{\sum_i v_i^2}}$
  * dot-product of normalized vectors - just multiply the numbers together, really
  
## Weighting schemes - document, query

* Now we can apply different weighting to documents and queries
* Typical choice:
  * document: log tf, no idf, length-normalized - why no idf?
  * query: log tf, log idf, length-normalized
  * cosine similarity

